In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mpl
from sklearn.metrics import r2_score

import glob
import os
import raytracing as rt
from scipy.ndimage import gaussian_filter

import scipy.optimize as spo
import scipy.constants as spc
import scipy.integrate as spi
import scipy.linalg as spl
import scipy.signal as spsi
import scipy.stats as spst

In [6]:
# mpl.plot(a=None, b=None,color="grey", label="Modèle ajusté de la polarisation TE")
# mpl.errorbar(error_a=None, error_b=None, yerr=11.1, xerr=0.02,fmt='o', color="grey", capsize=3, label="Données expérimentales de polarisation TE")

# mpl.ylabel("Intensité [µA]")
# mpl.xlabel("Angle du rayon incident [°]")
# mpl.legend(frameon=False) 
# mpl.tick_params(axis='both', which='both', direction='in')
# mpl.minorticks_on()

# mpl.show()


In [7]:
# Fonction pour calculer une moyenne
def moyenne_2D(liste, n, m):
    moyenne = []
    sous_moyenne = 0
    for i in range(n) :
        for j in range(m) :
            sous_moyenne += liste[i][j]
        moyenne += [sous_moyenne/(m)]
        sous_moyenne = 0
    return moyenne

In [8]:
popt_data, pcov_data = spo.curve_fit(fonction=None, variable_1=None, variable_2=None)
perr_data = np.sqrt(np.diag(pcov_data))

# Définir la fonction gaussienne
def gaussienne(x, a, b, c):
    return a * np.exp(-(x - b)**2 / (2 * c**2))
# Ajustement de la courbe gaussienne aux données
popt, pcov = spo.curve_fit(gaussienne, x_data=None, y_data=None, 
                           p0=[max(y_data=None), np.mean(x_data=None), np.std(x_data=None)])


def poisson_distribution(k, lambd):
    return spst.poisson.pmf(k, lambd)
# Ajustez le modèle de distribution de Poisson aux données
params, _ = spo.curve_fit(poisson_distribution, valeurs=None, counts=None, p0=[5])

TypeError: curve_fit() missing 3 required positional arguments: 'f', 'xdata', and 'ydata'

In [ ]:
def extract_data(path):
    """Fonction qui lit les données présentes dans des fichiers.

    Args:
        path (folder): Le path du dossier où sont les fichiers à lire.

    Returns:
        DataFrame: un dataframe de toutes les données avec une colonne par fichier.
    """
    all_files = glob.glob(os.path.join(path))
    dfs = pd.DataFrame()
    i = 0
    for f in all_files:
        dfs[i] = pd.read_csv(f)
        i += 1
    return dfs

In [ ]:
def extract_data(path):
    """Fonction qui sort les données voulues de fichier ".txt".

    Args:
        path (path): chemin d'accès au dossier qui contient les fichiers.

    Returns:
        array: un array de dataframe contenant les données voulues.
    """
    data = []
    # Parcourir les fichiers dans le dossier
    for filename in os.listdir(path):
        if filename.endswith(".txt"):
            filepath = os.path.join(path, filename)
        
            # Ouvrir le fichier et lire les données
            with open(filepath, 'r') as f:
                lines = f.readlines()
                start_index = lines.index(">>>>>Begin Spectral Data<<<<<\n") + 1
                spectral_data = [line.strip().split('\t') for line in lines[start_index:]]
                spectral_data = pd.DataFrame(spectral_data, columns=[0, 1])
                # Conversion des valeurs en float
                spectral_data[0] = spectral_data[0].str.replace(',', '.').astype(float)
                spectral_data[1] = spectral_data[1].str.replace(',', '.').astype(float)
                spectral_data["Filename"] = filename
                data.append(spectral_data)

    # Concaténer tous les DataFrames en un seul dataset
    return data

In [ ]:
def signaltonoise(dataset, min, max, axis=0, ddof=0):
    """Fonction qui détermine le SNR d'un groupe de données.

    Args:
        datset (dataframe): Données.
        min (float) : valeur  minimale pour la normalisation.
        max (float) : Valeur maximale pour la normalisation.    
        axis (int, optional): Axis along which the means are computed. Default is 0.
        ddof (float, int, optional): Delta degrees of freedom. The divisor used in the calculation is N - ddof, where N represents the number of elements.

    Returns:
        ndarray: Signal to Noise Ratio for each sample of the dataset.
    """
    SNR = []
    for df in dataset:
        signal = []
        subset = df.loc[(df[0] <= min) & (df[0] >= max)]
        m = subset[1].mean()
        sd = subset[1].std()
        noise = np.full(len(df), m)
        for i in range(len(df)):
            signal += [df[1][i]-noise]
        SNR += [signal/noise]
    
    return SNR

In [ ]:
def moyenne_SNR(dataframe):
    """Fonction qui fait la moyenne des données des lignes d'un dataframe.

    Args:
        dataframe (DataFrame): Dataframe de plusieurs colonnes

    Returns:
        DataFrame: DataFrame d'une seule colonne de la moyennes de toutes les lignes du dataframe précédent.
    """
    return dataframe.mean(axis=1)

In [ ]:
def normalisation(dataframe):
    """Fonction qui normalise les dataframes.

    Args:
        dataframe (DataFrame)

    Returns:
        DataFrame: Dataframe normalisé.
    """
    return (dataframe-min(dataframe))/np.ptp(dataframe)

In [ ]:
def normalize_dataframes(df_list):
    """Fonction qui normalise les données d'intensité fournies.

    Args:
        df_list (list): Liste de dataframes qui contiennent les données à normaliser.

    Returns:
        list: Liste de dataframes qui contiennent les données normalisées.
    """
    # Trouver le maximum global parmi tous les DataFrame
    max_value = -np.inf
    for df in df_list:
        local_max = np.max(df[1])
        if local_max > max_value:
            max_value = local_max
    
    # Normaliser chaque DataFrame sur 1 en utilisant le maximum global
    for df in df_list:
        df[1] = df[1] / max_value
    return df_list

In [ ]:
def extract_and_clean(path):
    """Fonction qui effectue les trois fonctions suivantes : normalisation, moyenne_SNR et extract_data

    Args:
        path (path): le path jusqu'au dossier des fichiers à lire.

    Returns:
        DataFrame: dataframe de 1 colonne normalisée.
    """
    return normalisation(moyenne_SNR(extract_data(path)))

In [ ]:
def moyenne_dataframes(liste_dataframes):
    """ 
    Retourne un DataFrame qui contient les moyennes des données de chaque dataframe de la liste.

    Args:
        - une liste de dataframes : list of pandas DataFrames

    Returns:
        - un DataFrame contenant les moyennes des données de chaque dataframe, sous la forme suivante :
            index   colonne1   colonne2   ...
            0       x         y          ...
    """
    # Fusionner les dataframes en une seule dataframe
    df_concat = pd.concat(liste_dataframes)

    # Regrouper les données par indice et calculer la moyenne pour chaque groupe
    donnees_regroupees = df_concat.groupby(df_concat.index)
    moyennes = donnees_regroupees.mean()
    # Retourner la liste des moyennes
    return moyennes

In [ ]:
def incertitude_dataframes(liste_dataframes):
    """ 
    Cette fonction prend en entrée une liste de dataframes et renvoie un dataframe unique qui contient les données des autres dataframes.

    Args:
        liste_dataframes (list of pandas DataFrame objects) : list of dataframes to be compared.

    Returns:
        float : the maximum incertitude among all dataframes in the list.
    """
    # Fusionner les dataframes en une seule dataframe
    df_concat = pd.concat(liste_dataframes)

    # Regrouper les données par indice et calculer l'incertitude pour chaque groupe
    donnees_regroupees = df_concat.groupby(df_concat.index)
    incertitude = donnees_regroupees["Intensity"].std()
    # Retourner la liste des incertitudes
    return incertitude

In [ ]:
def moyenne_incertitude_dataframe(liste_dataframe):
    """ 
    Cette fonction calcule la moyenne incertitude pour un dataframe. 

    Args:
        liste_dataframe (list of pandas DataFrame): La liste des données à traiter.

    Returns:
        pandas DataFrame : Le dataframe contenant les moyennes et incertitudes calculées.
    """
    moyenne = moyenne_dataframes(liste_dataframe)
    incertitude = incertitude_dataframes(liste_dataframe)
    return [moyenne, incertitude]

In [ ]:
def incertitude_moyenne(liste):
    """ 
        Calcul de l'incertitude moyenne d'une liste de nombres.

    Args:
        liste (list of float): La liste des nombres pour lesquels on veut calculer l'incertitude.

    Returns:
        float : L'incertitude moyenne de la liste.
    """
    moyenne_inc = []
    for elem in liste:
        moyenne_inc += [elem.mean()]
    return moyenne_inc

In [ ]:
def find_intensity(dataset, var, divisor=1):
    intensity = []
    for df in dataset:
        point = df.loc[df[0] == var]
        intensity += [np.max(point[1]/divisor)]
    return intensity

def find_maximum_intensity(dataset,mini=0, maxi=1, divisor=1):
    """Fonction qui trouve les maximums d'intensité.

    Args:
        dataset (array): dataset comprenant plusieurs dataframe.

    Returns:
        array: liste des maximums pour chaque dataframe.
    """
    max_from_dataset = []
    for df in dataset:
        subset = df.loc[(df[0] >= mini) & (df[0] <= maxi)]
        max_from_dataset += [np.max(subset[1]/divisor)]
    return max_from_dataset